# Reservas de Hotel 
*Elaborado por: Manuel Hernandez | Maria Alejandra Jaime | Matias Wajnman | Viridiana Valencia*



# Data Acquisition

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import pandas_profiling
% pip install pycountry
import pycountry as pc

UsageError: Line magic function `%` not found.


In [2]:
df = pd.read_csv('../primera_entrega/hotel_booking.csv')
df.head(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,email,phone-number,credit_card
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,Transient,0.0,0,0,Check-Out,2015-07-01,Ernest Barnes,Ernest.Barnes31@outlook.com,669-792-1661,************4322
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,Transient,0.0,0,0,Check-Out,2015-07-01,Andrea Baker,Andrea_Baker94@aol.com,858-637-6955,************9157
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,2015-07-02,Rebecca Parker,Rebecca_Parker@comcast.net,652-885-2745,************3734
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,2015-07-02,Laura Murray,Laura_M@gmail.com,364-656-8427,************5677
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,Transient,98.0,0,1,Check-Out,2015-07-03,Linda Hines,LHines@verizon.com,713-226-5883,************5498


# Data Wrangling

En primer lugar, se debe manipular la data para modificar o eliminar registros, bien sea porque no son relevantes al estudio, necesiten algún tipo de edición para su uso dentro del modelo predictivo, o presenten valores desconocidos que requieran ser imputados o eliminados, entre otros. Para ello, empezaremos por conocer los tipos de datos y las cantidades de registros existentes de cada feature del dataset.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 36 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

En base a la información vista, se realizarán una serie de modificaciones para mejorar la calidad del dataset. Ellas son:
* Se eliminarán las features agent y company, puesto que presentan una alta cantidad de valores missing.
* Se eliminarán las features name, email, phone-number y credit_card, debido a que al ser features que presentan distintos valores para cada cliente nuevo, estas variables no son de importancia para el modelo.
* Se imputaran valores para los registros en los cuales la variable children o country no presenten data. En el caso de children, se asume que la reservación no presenta niños así que se asigna el valor 0; mientras que en el caso de country se indica que el país no está disponible.
* Se eliminarán los registros en los cuales no hayan huéspedes adultos asociados pero existan huéspedes de otro tipo (children o babies).
* Se eliminarán los registros en los cuales la cantidad de huéspedes adultos es igual a cero.
* Se eliminarán los registros en los cuales el adr es menor a 0 o mayor a 400.
* Se eliminarán los registros en los cuales el tipo de habitacion sea P ó L.



* Se agregarán nuevas features: 
    * total_guests, el cual será la suma del total de huéspedes de tipo adults, children o babies asociados a cada reservación.
    * total_stay_in_nights, el cual será la suma del total de días, tanto de semana como de fín de semana, asociados a la reservación.
    * has_minors, el cual indica si hay huéspedes del tipo children o babies en la reserva (1) o no (0)
    * room_type_changed, el cual indica si el tipo de habitación asignada a la llegada al hotel es distinta al tipo de habitación reservada (1) o no (0).
    * cancelations_ratio, el cual indica si el huésped tiene una mayor cantidad de reservaciones previas canceladas (1) que de reservaciones no canceladas (0).

# Eliminar: agent, company, name, email, phone, credit card

In [4]:
prelim_df = df.copy()
prelim_df = prelim_df.drop(columns=["agent", "company", "name", "email", "phone-number", "credit_card"], axis=1)
prelim_df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,C,3,No Deposit,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,C,4,No Deposit,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,C,0,No Deposit,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,A,0,No Deposit,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,A,0,No Deposit,0,Transient,98.0,0,1,Check-Out,2015-07-03


# Valores Missing

In [5]:
features = prelim_df.columns.to_series()
nan_counts = prelim_df.count()
df_nan = pd.DataFrame(nan_counts).set_index(features)
filt = df_nan[0] != df.shape[0]
df_nan[filt]

,0
children,119386
country,118902


In [6]:
# Children
prelim_df.children.fillna(0,inplace=True)
df_nan = pd.DataFrame(prelim_df.count()).set_index(features)
df_nan[filt]

,0
children,119390
country,118902


In [7]:
# Country
prelim_df.country.fillna('Unavailable',inplace=True)
df_nan = pd.DataFrame(prelim_df.count()).set_index(features)
df_nan[filt]

,0
children,119390
country,119390


# Eliminar registros donde no hayan huéspedes adultos.

In [8]:
# Cambiamos el tipo de variable de children a int
prelim_df["children"] = prelim_df["children"].astype(int)

In [9]:
# Revisa si adultos == 0 y si es asi cambia adultos = 1 y niños = 0
no_adult_df = prelim_df[(prelim_df["adults"] == 0) & ((prelim_df["children"] != 0)|(prelim_df["babies"] != 0) )]["adults"]
print("Registros que no tienen huespedes adultos pero huespedes de otro tipo -> ", no_adult_df.count())
print("Dimension del dataframe actual -> ", prelim_df.shape)

Registros que no tienen huespedes adultos pero huespedes de otro tipo ->  223
Dimension del dataframe actual ->  (119390, 30)


In [10]:
## Elimina los registros que cumplen la condicion anterior
prelim_df = prelim_df.drop(no_adult_df.index)
print("Dimension del dataframe actual -> ", prelim_df.shape)

Dimension del dataframe actual ->  (119167, 30)


# Nuevas Features
## Feature total huespedes (total_guests)

In [11]:
prelim_df["total_guests"] = prelim_df["adults"] + prelim_df["children"] + prelim_df["babies"]
prelim_df.head().T

,0,1,2,3,4
hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel
is_canceled,0,0,0,0,0
lead_time,342,737,7,13,14
arrival_date_year,2015,2015,2015,2015,2015
arrival_date_month,July,July,July,July,July
arrival_date_week_number,27,27,27,27,27
arrival_date_day_of_month,1,1,1,1,1
stays_in_weekend_nights,0,0,0,0,0
stays_in_week_nights,0,0,1,1,2
adults,2,2,1,1,2


## Feature total días en el hotel (total_stay_in_nights)

In [12]:
prelim_df["total_stay_in_nights"] = prelim_df["stays_in_week_nights"] + prelim_df["stays_in_weekend_nights"]
prelim_df.head().T

,0,1,2,3,4
hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel
is_canceled,0,0,0,0,0
lead_time,342,737,7,13,14
arrival_date_year,2015,2015,2015,2015,2015
arrival_date_month,July,July,July,July,July
arrival_date_week_number,27,27,27,27,27
arrival_date_day_of_month,1,1,1,1,1
stays_in_weekend_nights,0,0,0,0,0
stays_in_week_nights,0,0,1,1,2
adults,2,2,1,1,2


In [13]:
# Cantidad de registros que tienen noches = 0 y que no fueron cancelados --- Son todos Check-Out
nfilt = (prelim_df["total_stay_in_nights"] == 0) & (prelim_df["is_canceled"] != 1)
prelim_df[nfilt]["reservation_status"].value_counts()

Check-Out    680
Name: reservation_status, dtype: int64

## Feature que indique si hay niños en la reserva (has_minors)

In [14]:
prelim_df["total_minors"] = prelim_df["children"] + prelim_df["babies"]
minors_filt = prelim_df["total_minors"] > 0
prelim_df[minors_filt]["total_minors"].value_counts()

1     5442
2     3567
3       97
10       2
9        1
Name: total_minors, dtype: int64

In [15]:
def hasMinors(minors):
    val = 0
    if (minors > 0):
        val = 1
    return val

prelim_df["total_minors"] = prelim_df["total_minors"].apply(hasMinors)
prelim_df = prelim_df.rename(columns={"total_minors": "has_minors"})
prelim_df.head(15).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel
is_canceled,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0
lead_time,342,737,7,13,14,14,0,9,85,75,23,35,68,18,37
arrival_date_year,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015
arrival_date_month,July,July,July,July,July,July,July,July,July,July,July,July,July,July,July
arrival_date_week_number,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27
arrival_date_day_of_month,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
stays_in_weekend_nights,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
stays_in_week_nights,0,0,1,1,2,2,2,2,3,3,4,4,4,4,4
adults,2,2,1,1,2,2,2,2,2,2,2,2,2,2,2


# Feature binario que compare el tipo de cuarto asignado con el tipo de cuarto reservado e indique si este cambió (room_type_changed)

In [16]:
# Asumo que para todos los registros assigned_room_type y reserved_room_type son iguales, asignandole cero inicialmente
prelim_df["room_type_changed"] = 0 
room_filt = (prelim_df["assigned_room_type"] != prelim_df["reserved_room_type"])
prelim_df.loc[room_filt, "room_type_changed"] = 1
prelim_df["room_type_changed"].value_counts()

0    104282
1     14885
Name: room_type_changed, dtype: int64

## Feature binario que indique si el cliente ha tenido más cancelaciones que estadías (cancelations_ratio)

In [17]:
prelim_df['cancelations_ratio'] = 0
prelim_df.loc[ prelim_df['previous_cancellations'] > prelim_df['previous_bookings_not_canceled'] , 'cancelations_ratio'] = 1

In [18]:
prelim_df.head(10).T

,0,1,2,3,4,5,6,7,8,9
hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel
is_canceled,0,0,0,0,0,0,0,0,1,1
lead_time,342,737,7,13,14,14,0,9,85,75
arrival_date_year,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015
arrival_date_month,July,July,July,July,July,July,July,July,July,July
arrival_date_week_number,27,27,27,27,27,27,27,27,27,27
arrival_date_day_of_month,1,1,1,1,1,1,1,1,1,1
stays_in_weekend_nights,0,0,0,0,0,0,0,0,0,0
stays_in_week_nights,0,0,1,1,2,2,2,2,3,3
adults,2,2,1,1,2,2,2,2,2,2


# Eliminaciones

## Registros donde la cantidad de Adultos es igual a cero

In [19]:
prelim_df[(prelim_df["adults"] == 0)]["adults"].count()

180

In [20]:
adults_filt = (prelim_df["adults"] == 0)
prelim_df = prelim_df.drop(prelim_df[adults_filt].index)
prelim_df[(prelim_df["adults"] == 0)]["adults"].count()

0

## Registros con adr negativo o adr mayor a 400

In [21]:
adr_filt = (prelim_df["adr"] < 0) | (prelim_df["adr"] > 300)
len(prelim_df[adr_filt])

277

In [22]:
print("Valor máximo de ADR en el dataset --> ", prelim_df["adr"].max())
print("Valor mínimo de ADR en el dataset --> ", prelim_df["adr"].min())

Valor máximo de ADR en el dataset -->  5400.0
Valor mínimo de ADR en el dataset -->  -6.38


In [23]:
adr_filt = (prelim_df["adr"] < 0) | (prelim_df["adr"] > 300)
prelim_df = prelim_df.drop(prelim_df[adr_filt].index)
print("Valor máximo de ADR en el dataset --> ", prelim_df["adr"].max())
print("Valor mínimo de ADR en el dataset --> ", prelim_df["adr"].min())

Valor máximo de ADR en el dataset -->  300.0
Valor mínimo de ADR en el dataset -->  0.0


## Registros con tipos de habitacion P y L

In [24]:
assigned_filt = (prelim_df["assigned_room_type"] == 'P') | (prelim_df["assigned_room_type"] == 'L')
reserved_filt = (prelim_df["reserved_room_type"] == 'P') | (prelim_df["reserved_room_type"] == 'L')
prelim_df[assigned_filt | reserved_filt][["assigned_room_type", "reserved_room_type"]].value_counts()

assigned_room_type  reserved_room_type
A                   L                     1
B                   L                     1
C                   L                     1
F                   L                     1
H                   L                     1
L                   L                     1
dtype: int64

In [25]:
prelim_df = prelim_df.drop(prelim_df[assigned_filt | reserved_filt].index)
len(prelim_df[assigned_filt | reserved_filt][["assigned_room_type", "reserved_room_type"]])

0

# Modificación de tipo de datos 
## Features is_canceled, is_repeated_guest, more_cancelations, room_type_changed y has_minors se convierten a tipo object

In [26]:
prelim_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118704 entries, 0 to 119389
Data columns (total 35 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           118704 non-null  object 
 1   is_canceled                     118704 non-null  int64  
 2   lead_time                       118704 non-null  int64  
 3   arrival_date_year               118704 non-null  int64  
 4   arrival_date_month              118704 non-null  object 
 5   arrival_date_week_number        118704 non-null  int64  
 6   arrival_date_day_of_month       118704 non-null  int64  
 7   stays_in_weekend_nights         118704 non-null  int64  
 8   stays_in_week_nights            118704 non-null  int64  
 9   adults                          118704 non-null  int64  
 10  children                        118704 non-null  int32  
 11  babies                          118704 non-null  int64  
 12  meal            

In [27]:
prelim_df[['is_canceled', 'is_repeated_guest', 'cancelations_ratio', 'room_type_changed', 'has_minors']] = prelim_df[['is_canceled', 'is_repeated_guest', 'cancelations_ratio', 'room_type_changed', 'has_minors']].astype('object')
prelim_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118704 entries, 0 to 119389
Data columns (total 35 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           118704 non-null  object 
 1   is_canceled                     118704 non-null  object 
 2   lead_time                       118704 non-null  int64  
 3   arrival_date_year               118704 non-null  int64  
 4   arrival_date_month              118704 non-null  object 
 5   arrival_date_week_number        118704 non-null  int64  
 6   arrival_date_day_of_month       118704 non-null  int64  
 7   stays_in_weekend_nights         118704 non-null  int64  
 8   stays_in_week_nights            118704 non-null  int64  
 9   adults                          118704 non-null  int64  
 10  children                        118704 non-null  int32  
 11  babies                          118704 non-null  int64  
 12  meal            

# Reinicio de los indices post Data Wrangling

In [28]:
data = prelim_df.copy()
data.reset_index()
print('Dimensiones iniciales --> ', df.shape)
print('Dimensiones luego de Data Wrangling --> ', data.shape)
print('Registros Eliminados --> ', df.shape[0]- data.shape[0])
print('Registros Eliminados (%) --> ', (df.shape[0]- data.shape[0])*100/df.shape[0])

Dimensiones iniciales -->  (119390, 36)
Dimensiones luego de Data Wrangling -->  (118704, 35)
Registros Eliminados -->  686
Registros Eliminados (%) -->  0.5745874863891448


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118704 entries, 0 to 119389
Data columns (total 35 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           118704 non-null  object 
 1   is_canceled                     118704 non-null  object 
 2   lead_time                       118704 non-null  int64  
 3   arrival_date_year               118704 non-null  int64  
 4   arrival_date_month              118704 non-null  object 
 5   arrival_date_week_number        118704 non-null  int64  
 6   arrival_date_day_of_month       118704 non-null  int64  
 7   stays_in_weekend_nights         118704 non-null  int64  
 8   stays_in_week_nights            118704 non-null  int64  
 9   adults                          118704 non-null  int64  
 10  children                        118704 non-null  int32  
 11  babies                          118704 non-null  int64  
 12  meal            

# Algoritmos de Clasificacion

### Empezamos por importar las librerias necesarias para llevar a cabo el estudio.

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

### Creamos una copia de la data para su uso en el modelo.

In [31]:
ml_data = data.copy()
ml_data.head(3)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,total_guests,total_stay_in_nights,has_minors,room_type_changed,cancelations_ratio
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,0.0,0,0,Check-Out,2015-07-01,2,0,0,0,0
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,0.0,0,0,Check-Out,2015-07-01,2,0,0,0,0
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,75.0,0,0,Check-Out,2015-07-02,1,1,0,1,0


### Como coexisten variables categóricas y numéricas en el dataset, usaremos la función get_dummies de pandas para transformar las variables categóricas a numéricas. 

In [32]:
ml_data = pd.get_dummies(ml_data, drop_first=True)
ml_data.shape

(118704, 1175)

### Establecemos las variables X y Y que serán pasadas al modelo. En este caso, se elimina del dataset la variable is_canceled_1 por ser nuestro target.

In [33]:
Y = ml_data["is_canceled_1"]
X = ml_data.drop("is_canceled_1", axis=1)

### Separamos la data en los grupos de testeo y de entrenamiento.

In [34]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=43)

### Instanciamos el arbol de decisión. Se utilizará una profundidad máxima de 2 y un random_state de 43.

In [35]:
first_tree = DecisionTreeClassifier(max_depth=2, random_state=43)

### Realizamos el entrenamiento del modelo a partir de los datos de entrenamiento.

In [36]:
first_tree.fit(X_train, Y_train)

DecisionTreeClassifier(max_depth=2, random_state=43)

### Realizamos la prediccion

In [37]:
y_train_pred = first_tree.predict(X_train) #Prediccion en Train
y_test_pred = first_tree.predict(X_test) #Prediccion en Test

### Probamos la precisión del modelo

In [38]:
train_accuracy = accuracy_score(Y_train, y_train_pred)
test_accuracy = accuracy_score(Y_test, y_test_pred)

print('% de aciertos sobre el set de entrenamiento:', train_accuracy)
print('% de aciertos sobre el set de evaluación:',test_accuracy)

% de aciertos sobre el set de entrenamiento: 1.0
% de aciertos sobre el set de evaluación: 1.0


### Los resultados indican que el modelo presenta overfitting , puesto que el modelo aprende la data en vez de reconocer patrones predictivos. Esto es comun en este tipo de algorítmos y ocurre cuando el arbol se genera de forma tal que representa perfectamente la data de entrenamiento. Por ende, termina con ramas con reglas estrictas basadas en data dispersa. Esto afecta la precisión de muestras que estan separadas de la data de entrenamiento 

### Para mejorar el modelo, se eliminarán algunos features del dataset para así reducir la dimensionalidad de la data, puesto que al transformar la data categórica a numérica se obtienen demasiadas columnas, ya que 35 columnas se transformaron en 1175 columnas dummy. Esto resulta, como fue explicado anteriormente, en reglas muy específicas que no contribuyen a obtener mejores resultados generales para la data que se usará para el testeo.

### Se utilizarán dos criterios en particular para la eliminación de features del dataset a modelar. En primer lugar, se eliminan features categóricos que presenten alta cardinalidad, como es el caso de country o reservation_status_date, los cuales presentan muchos valores distintos. También se eliminan features que guardan relación estrecha con otros features del dataset, como es el caso de has_minors que guarda estrecha relación con la cantidad de bebes y niños asociados a cada reservación, o el caso de reservation_status en el cual ciertas categorias estan compuestas mayoritariamente de reservaciones canceladas o no canceladas.

In [39]:
ml_data_2 = data.copy()
dropped_features_list = ["arrival_date_month", "country", "reservation_status_date", "is_repeated_guest", "reserved_room_type", "reservation_status", "has_minors", "room_type_changed", "cancelations_ratio"]
ml_data_2 = ml_data_2.drop(dropped_features_list, axis=1)
ml_data_2.head(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,total_guests,total_stay_in_nights
0,Resort Hotel,0,342,2015,27,1,0,0,2,0,...,C,3,No Deposit,0,Transient,0.0,0,0,2,0
1,Resort Hotel,0,737,2015,27,1,0,0,2,0,...,C,4,No Deposit,0,Transient,0.0,0,0,2,0
2,Resort Hotel,0,7,2015,27,1,0,1,1,0,...,C,0,No Deposit,0,Transient,75.0,0,0,1,1
3,Resort Hotel,0,13,2015,27,1,0,1,1,0,...,A,0,No Deposit,0,Transient,75.0,0,0,1,1
4,Resort Hotel,0,14,2015,27,1,0,2,2,0,...,A,0,No Deposit,0,Transient,98.0,0,1,2,2


In [41]:
ml_data_2 = pd.get_dummies(ml_data_2, drop_first=True)
ml_data_2.shape

(118704, 49)

### Se observa que al utilizar la función get_dummies se obtiene una cantidad de columnas mucho menor que las obtenidas en la data utilizada en el primer arbol. Por ende se procede a utilizar esta data en un nuevo arbol de decisión, nuevamente extrayendo del dataset el target (is_canceled_1), y realizando un split entre data de entrenamiento y de test con los mismos parámetros del modelo anterior.

In [42]:
Y2 = ml_data_2["is_canceled_1"]
X2 = ml_data_2.drop("is_canceled_1", axis=1)

In [43]:
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, Y2, test_size=0.3, random_state=43)

In [44]:
second_tree = DecisionTreeClassifier(max_depth=2, random_state=43)
second_tree.fit(X2_train, Y2_train)

DecisionTreeClassifier(max_depth=2, random_state=43)

### Realizamos predicciones para la data de entrenamiento y de test, y revisamos su precisión a través del accuracy score.

In [45]:
y2_train_pred = second_tree.predict(X2_train) 
y2_test_pred = second_tree.predict(X2_test) 

In [46]:
train_accuracy_2 = accuracy_score(Y2_train, y2_train_pred)
test_accuracy_2 = accuracy_score(Y2_test, y2_test_pred)

print('% de aciertos sobre el set de entrenamiento:', train_accuracy_2)
print('% de aciertos sobre el set de evaluación:',test_accuracy_2)

% de aciertos sobre el set de entrenamiento: 0.7501805227940115
% de aciertos sobre el set de evaluación: 0.7509547343592048


### Como se puede observar, el descartar ciertos features del dataset permite obtener un mejor modelo, que cuente con menor especificidad y permita hacer una generalización mas realista de los datos.

### A modo de prueba, se pueden agregar mas niveles al arbol de decisiones y así ver el efecto que tiene este parámetro sobre la precisión del modelo para la misma data.

In [47]:
third_tree = DecisionTreeClassifier(max_depth=5, random_state=43)

In [48]:
third_tree.fit(X2_train, Y2_train)

DecisionTreeClassifier(max_depth=5, random_state=43)

In [49]:
y3_train_pred = third_tree.predict(X2_train) 
y3_test_pred = third_tree.predict(X2_test) 

In [50]:
train_accuracy_3 = accuracy_score(Y2_train, y3_train_pred)
test_accuracy_3 = accuracy_score(Y2_test, y3_test_pred)

print('% de aciertos sobre el set de entrenamiento:', train_accuracy_3)
print('% de aciertos sobre el set de evaluación:',test_accuracy_3)

% de aciertos sobre el set de entrenamiento: 0.8039522457035575
% de aciertos sobre el set de evaluación: 0.8038301696057508


### Por último, aumentaremos de manera significativa la profundidad del arbol, para así observar en que medida mejora la precisión (max_depth = 25) del modelo respecto a un aumento menor.

In [51]:
fourth_tree = DecisionTreeClassifier(max_depth=25, random_state=43)
fourth_tree.fit(X2_train, Y2_train)
y4_train_pred = fourth_tree.predict(X2_train) 
y4_test_pred = fourth_tree.predict(X2_test) 
train_accuracy_4 = accuracy_score(Y2_train, y4_train_pred)
test_accuracy_4 = accuracy_score(Y2_test, y4_test_pred)

print('% de aciertos sobre el set de entrenamiento:', train_accuracy_4)
print('% de aciertos sobre el set de evaluación:',test_accuracy_4)

% de aciertos sobre el set de entrenamiento: 0.9611514947287344
% de aciertos sobre el set de evaluación: 0.8213523531393913


In [53]:
accuracy_difference = abs(test_accuracy_4 - test_accuracy_3) *100 / test_accuracy_3
print('Diferencia porcentual entre modelos para la data de testeo ----> ', round(accuracy_difference,2), '%')

Diferencia porcentual entre modelos para la data de testeo ---->  2.18 %


### Se observa que al utilizar una mayor profundidad en el arbol de decisión mejora la precisión del modelo. Sin embargo, a pesar de que se pueden agregar mucha más profundidad al modelo, se observa que a partir de cierta cantidad de niveles extra, el aumento en la precisión del mismo es irrisorio en contraste al aumento en la complejidad. También se puede observar que esta variación en la profundidad resulta en una mayor precisión para la data de entrenamiento, que se explica en una mayor cantidad de ramas con alta especificidad, resultando en un overfitting del modelo.